In [1]:
from italian_csv_type_prediction.dataframe_generators import SimpleDatasetGenerator
from italian_csv_type_prediction.models import TypePredictor
from sklearn.metrics import accuracy_score, balanced_accuracy_score

In [2]:
generator = SimpleDatasetGenerator()

In [3]:
x_train, y_train = generator.build(number=100_000, chunk_size=100)

In [4]:
x_test, y_test = generator.build(number=5000, chunk_size=10)

In [5]:
model = TypePredictor()

model.fit(x_train, y_train)

/usr/local/lib/python3.6/dist-packages/sklearn/base.py:334: UserWarning: Trying to unpickle estimator DecisionTreeClassifier from version 0.23.1 when using version 0.23.2. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)


In [6]:
y_pred = model._model.predict(x_test)
y_train_pred = model._model.predict(x_train)

In [7]:
accuracy_score(y_test, y_pred), balanced_accuracy_score(y_test, y_pred)

(0.9976880392756053, 0.9956006418693033)

In [8]:
accuracy_score(y_train, y_train_pred), balanced_accuracy_score(y_train, y_train_pred)

(0.9977994259747273, 0.9957942622670488)

In [9]:
X, y = SimpleDatasetGenerator().generate_simple_dataframe(max_rows=20)

In [10]:
y

,ItalianFiscalCode,CadastreCode,Document,Tax,Plate,Address,ItalianZIPCode,ProvinceCode,Region,Municipality,...,String,EMail,PhoneNumber,Date,BiologicalSex,Boolean,Name,Surname,SurnameName,NameSurname
0,ItalianFiscalCode,CadastreCode,Document,Tax,Plate,Address,ItalianZIPCode,ProvinceCode,Region,Municipality,...,String,EMail,PhoneNumber,Date,BiologicalSex,Boolean,Name,Surname,SurnameName,NameSurname
1,ItalianVAT,CadastreCode,Document,Tax,Plate,NaN,ItalianZIPCode,ProvinceCode,Region,Municipality,...,String,EMail,PhoneNumber,Date,BiologicalSex,Boolean,Company,Company,Company,Company
2,ItalianFiscalCode,CadastreCode,Document,Tax,Plate,Address,ItalianZIPCode,ProvinceCode,Region,Municipality,...,String,EMail,PhoneNumber,Date,BiologicalSex,Boolean,Name,Surname,NaN,NameSurname
3,ItalianFiscalCode,CadastreCode,Document,Tax,Plate,Address,ItalianZIPCode,ProvinceCode,Region,Municipality,...,String,EMail,PhoneNumber,Date,BiologicalSex,Boolean,Name,Surname,SurnameName,NameSurname
4,ItalianVAT,CadastreCode,Document,Tax,Plate,Address,ItalianZIPCode,ProvinceCode,Region,Municipality,...,String,EMail,PhoneNumber,Date,BiologicalSex,Boolean,Company,NaN,Company,Company
5,ItalianVAT,CadastreCode,Document,Tax,Plate,Address,ItalianZIPCode,ProvinceCode,Region,Municipality,...,String,EMail,PhoneNumber,Date,BiologicalSex,Boolean,Company,Company,Company,Company
6,ItalianVAT,CadastreCode,Document,Tax,Plate,Address,ItalianZIPCode,ProvinceCode,Region,Municipality,...,NaN,EMail,PhoneNumber,Date,BiologicalSex,Boolean,Company,Company,Company,Company
7,ItalianVAT,CadastreCode,Document,Tax,Plate,Address,ItalianZIPCode,ProvinceCode,Region,Municipality,...,String,EMail,PhoneNumber,Date,BiologicalSex,Boolean,Company,Company,Company,Company
8,ItalianFiscalCode,CadastreCode,NaN,Tax,Plate,Address,ItalianZIPCode,ProvinceCode,Region,Municipality,...,String,EMail,PhoneNumber,Date,BiologicalSex,Boolean,Name,Surname,SurnameName,NameSurname
9,ItalianFiscalCode,CadastreCode,Document,Tax,Plate,Address,ItalianZIPCode,ProvinceCode,Region,Municipality,...,NaN,EMail,PhoneNumber,NaN,BiologicalSex,Boolean,Name,Surname,SurnameName,NameSurname


In [11]:
model.predict_dataframe(X)

[INFO] 2021-01-31 09:18:09,955 : Transforming given DataFrame to embedding.
[INFO] 2021-01-31 09:26:14,915 : Executing predictions on given DataFrame.
[INFO] 2021-01-31 09:26:14,918 : Executing reverse transformation of the predictions from embedding.


,ItalianFiscalCode,CadastreCode,Document,Tax,Plate,Address,ItalianZIPCode,ProvinceCode,Region,Municipality,...,String,EMail,PhoneNumber,Date,BiologicalSex,Boolean,Name,Surname,SurnameName,NameSurname
0,ItalianFiscalCode,CadastreCode,Document,Tax,Plate,Address,ItalianZIPCode,ProvinceCode,Region,Municipality,...,String,EMail,PhoneNumber,Date,BiologicalSex,Boolean,Name,Surname,SurnameName,NameSurname
1,ItalianVAT,CadastreCode,Document,Tax,Plate,NaN,ItalianZIPCode,ProvinceCode,Region,Municipality,...,String,EMail,PhoneNumber,Date,BiologicalSex,Boolean,Company,Company,Company,Company
2,ItalianFiscalCode,CadastreCode,Document,Tax,Plate,Address,ItalianZIPCode,ProvinceCode,Region,Municipality,...,String,EMail,PhoneNumber,Date,BiologicalSex,Boolean,Name,Surname,NaN,NameSurname
3,ItalianFiscalCode,CadastreCode,Document,Tax,Plate,Address,ItalianZIPCode,ProvinceCode,Region,Municipality,...,String,EMail,PhoneNumber,Date,BiologicalSex,Boolean,Name,Surname,SurnameName,NameSurname
4,ItalianVAT,CadastreCode,Document,Tax,Plate,Address,ItalianZIPCode,ProvinceCode,Region,Municipality,...,String,EMail,PhoneNumber,Date,BiologicalSex,Boolean,Company,NaN,Company,Company
5,ItalianVAT,CadastreCode,Document,Tax,Plate,Address,ItalianZIPCode,ProvinceCode,Region,Municipality,...,String,EMail,PhoneNumber,Date,BiologicalSex,Boolean,Company,Company,Company,Company
6,ItalianVAT,CadastreCode,Document,Tax,Plate,Address,ItalianZIPCode,ProvinceCode,Region,Municipality,...,NaN,EMail,PhoneNumber,Date,BiologicalSex,Boolean,Company,Company,Company,Company
7,ItalianVAT,CadastreCode,Document,Tax,Plate,Address,ItalianZIPCode,ProvinceCode,Region,Municipality,...,String,EMail,PhoneNumber,Date,BiologicalSex,Boolean,Company,Company,Company,Company
8,ItalianFiscalCode,CadastreCode,NaN,Tax,Plate,Address,ItalianZIPCode,ProvinceCode,Region,Municipality,...,String,EMail,PhoneNumber,Date,BiologicalSex,Boolean,Name,Surname,SurnameName,NameSurname
9,ItalianFiscalCode,CadastreCode,Document,Tax,Plate,Address,ItalianZIPCode,ProvinceCode,Region,Municipality,...,NaN,EMail,PhoneNumber,NaN,BiologicalSex,Boolean,Name,Surname,SurnameName,NameSurname


In [12]:
from collections import Counter

mask = y_test != y_pred

true_labels = model._embedder._encoder.inverse_transform(y_test[mask])
predicted_labels = model._embedder._encoder.inverse_transform(y_pred[mask])

Counter(zip(true_labels, predicted_labels))

Counter({('NaN', 'CountryCode'): 353,
         ('Error', 'Document'): 993,
         ('Error', 'Company'): 1361,
         ('Company', 'Error'): 3598,
         ('Company', 'NaN'): 764,
         ('CountryCode', 'NaN'): 16,
         ('Error', 'ProvinceCode'): 30,
         ('Error', 'PhoneNumber'): 49,
         ('Error', 'ItalianZIPCode'): 106,
         ('Float', 'Integer'): 167,
         ('Error', 'CountryCode'): 21,
         ('ItalianZIPCode', 'Integer'): 97,
         ('String', 'Company'): 42,
         ('Float', 'NaN'): 10,
         ('NaN', 'Company'): 16,
         ('NameSurname', 'String'): 8,
         ('ItalianZIPCode', 'Error'): 47,
         ('ItalianVAT', 'Error'): 7,
         ('Integer', 'NaN'): 11,
         ('CountryCode', 'ProvinceCode'): 5,
         ('Error', 'ItalianFiscalCode'): 2,
         ('Error', 'NaN'): 4,
         ('Company', 'String'): 75,
         ('Surname', 'String'): 4,
         ('SurnameName', 'String'): 4,
         ('Error', 'String'): 15,
         ('Error', 'Boole